In [21]:
import os
import requests
import zipfile

from bs4 import BeautifulSoup

url = 'https://www.ojk.go.id/id/kanal/perbankan/data-dan-statistik/statistik-perbankan-indonesia/Pages/Statistik-Perbankan-Indonesia---Juli-2020.aspx'
root = 'https://www.ojk.go.id/'

r = requests.get(url)
soup = BeautifulSoup(r.text, 'html.parser')

all_hrefs = soup.find_all('a')
all_links = [link.get('href') for link in all_hrefs]
zip_files = [dl for dl in all_links if dl and '.zip' in dl]
download_folder = 'output'

In [23]:
if not os.path.exists(download_folder):
    os.makedirs(download_folder)

tries = 0
for zip_file in zip_files:
    full_url = root + zip_file
    zip_filename = os.path.basename(zip_file)
    dl_path = os.path.join(download_folder, zip_filename)
    if os.path.exists(dl_path):
        # you have already downloaded this file, so skip it
        continue

    while tries < 3:
        r = requests.get(full_url)
        dl_path = os.path.join(download_folder, zip_filename)
        with open(dl_path, 'wb') as z_file:
            z_file.write(r.content)

        # unzip the file
        extract_dir = os.path.splitext(os.path.basename(zip_file))[0]
        try:
            z = zipfile.ZipFile(dl_path)
            z.extractall(os.path.join(download_folder, extract_dir))
            break
        except zipfile.BadZipfile:
            # the file didn't download correctly, so try again
            # this is also a good place to log the error
            pass
        tries += 1